Link to paper: [Zenke, Ganguli - 2018](https://direct.mit.edu/neco/article/30/6/1514-1541/8378)

Zenke's [Tutorial](https://github.com/fzenke/spytorch) on Surrogate Gradient Descent using PyTorch.

To Implement:
1. LIF Neurons (maybe a class of such neurons)
2. Fast Sigmoid Function

Question:
1. How to implement spiking neural network in pytorch? 
  * use RNNs as Zenke suggests in his tutorial?

In [1]:
#@title Dependencies
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dtype = torch.float
device = torch.device("cpu")

# Uncomment the line below to run on GPU
# device = torch.device("cuda:0") 

### Network Architecture

3 layer feed-forward neural network

In [ ]:
nb_inputs  = 100
nb_hidden  = 4
nb_outputs = 2

In [ ]:
batch_size = 256

### Spiking Neuron Model Setup

Since we are technically stimulating an RNN, the neurons have to be simulated for a certain number of timesteps

In [ ]:
time_step = 1e-3
nb_steps  = 200

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

Initializing weights from a normal distribution, the variance is scaled with the inverse square root of the number of input connections.

Dale's Law is ignored here.

In [ ]:
#@title Weight Matrcies
weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

w1 = torch.empty((nb_inputs, nb_hidden),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w1, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))

w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w2, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))

print("init done")

In [ ]:
#@title The Spiking Non-linearity
def spike_fn(x):
  out = torch.zeros_like(x)
  out[x > 0] = 1.0
  return out

In [ ]:
h1 = torch.einsum("abc,cd->abd", (x_data, w1))

Initialize the synaptic currents and the membrane potentials at zero. Then implement a loop that stimulates the neuron models over time, and record the membrane potential and output spikes of all trials and all neurons.

In [ ]:
# tensors initialized with zeros for synaptic current and membrane potential
syn = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
mem = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)

# two lists to record the membrane potentials and output spikes
mem_rec = []
spk_rec = []

# The simulation loop
for t in range(nb_steps):
  m_thr = mem - 1.0
  out = spike_fn(m_thr)
  rst = out.detach() # we do not want to backprop through the reset

  new_syn = alpha*syn + h1[:, t]
  new_mem = (beta*mem + syn)(1 - rst)

  mem_rec.append(mem)
  spk_rec.append(out)

  mem = new_mem
  syn = new_syn

mem_rec = torch.stack(mem_rec, dim=1)
spk_rec = torch.stack(spk_rec, dim=1)




In [ ]:
def run_snn(inputs):

  h1 = torch.einsum('abc,cd->abd', (inputs,w1))
  syn = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
  mem = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
  # lists to record the membrane potentials and the synaptic currents:
  mem_rec = []
  spk_rec = []
  # loop to simulate time
  for t in range(nb_steps):
    mthr = mem - 1.0
    out = spike_fn(mthr)
    rst = out.detach()  # do not want to backpropagate through reset

    new_syn = alpha*syn + h1[:, t]
    new_mem = (beta*mem + syn)(1 - rst)

    mem_rec.append(mem)
    spk_rec.append(out)

    mem = new_mem
    syn = new_syn
  
  # create tensors to stack the elements in the recording lists
  mem_rec = torch.stack(mem_rec, dim=1)
  spk_rec = torch.stack(spk_rec, dim=1)

  # readout layer
  h2 = torch.einsum('abc,cd->abd', (spk_rec, w2))
  flt = torch.zeros((batch_size, nb_outputs), device=device, dtype=dtype)
  out = torch.zeros((batch_size, nb_outputs), device=device, dtype=dtype)
  out_rec = [out]
  for t in range(nb_steps):
    new_flt = alpha*flt + h2[:, t]
    new_out = beta*out + flt

    flt = new_flt
    out = new_out

    out_rec.append(out)

  out_rec = torch.stack(out_rec, dim=1)
  other_recs = [mem_rec, spk_rec]
  return out_rec, other_recs


### LIF Neuron


## SuperSpike Implementation

In [ ]:
class SuperSpike(nn.Module):
  def.__init__(self):
    super(SuperSpike, self).__init__()
    self.layers = nn.Sequential()

  def forward(self, x):
    i



In [ ]:
#@title Surrogate Gradient

class SurrGradSpike(torch.autograd.Function):

  scale = 100.0 # controls the steepness of the gradient

  @staticmethod
  def forward(ctx, input):
    '''
    computes a step-function on the input. ctx is a context variable
    that stores information needed later for backpropagation
    '''
    ctx.save_for_backward(input)
    out = torch.zeros_like(input)
    out[input > 0] = 1
    return out

  @staticmethod
  def backward(ctx, grad_output):
    '''
    In the backward method, we recieve a tensor we need to compute 
    the surrogradient of the loss with respect to the input. 
    Here we use the negative half of the fast sigmoid as in 
    Zenke & Ganguli 2018.
    
    '''
    input, _ = ctx.saved_tensors
    grad_input = grad_output.clone()
    grad = grad_input/(SurrGradSpike.scale*torch.abs(input)+1.0)**2
    return grad

# overwrite the spike function with the surrograte gradient function
# using the apply method
spike_fn = SurrGradSpike.apply


#### Loss Function

The van rossum distance is evaluated as:
$$ L = (\alpha*S_i - \alpha*\hat{S_i})^2 $$

where $ \alpha $ is a double exponential filter. 

### Double Exponential Filter




In [ ]:

def doubleExponential(spike_train, dt, t_rise, t_decay, time):
  """
  Implements the double exponential kernel
  input:
    spike_train - time series of spikes containing ones and zeros
    dt
    t_rise - time constant of first exponential filter
    t_decay - time constant of second exponential filter
    time - tuple with start & stop time (wrt spike_train) for the convolution
  Returns:
    the convolved double exponential product
  """
  time_range = time[1] - time[0]

  z = torch.zeros(time_range)
  z_hat = torch.zeros(time_range)

  for t in range(time_range):
    z[t+1] =  z[t] + (-z[t] + spike_train[t])*dt
    z_hat[t+1] = z_hat[t] + (-z_hat[t] + z[t])*dt

  return z_hat

### Hebbian Coincidence Detection & Synaptic Eligibility Trace

$$ \frac{dw_{ij}}{dt} = r\int_{-\infty}^t ds\ e_i(s)\ \alpha * [\sigma'(U_i(s))(\epsilon*S_j(s))]  $$

The evalutation of this equation requires:
1. evaluation of presynaptic traces
2. evaluation of hebbian coincidence and computation of the synaptic eligibility traces
3. compuatation and propagation of error signals
4. integration of this equation and weight update



Here $ \lambda_{ij} = \sigma'(U_i(s))(\epsilon*S_j(s)) $ is the eligibility trace.

Fast sigmoid: $$   \sigma(x) = \frac{x}{1 + |x|} $$

So, $ \sigma'(U_i) = \frac{1}{(1 + |h_i|)^2} $

where $ h_i = \beta(U_i - \nu) $


In [ ]:
def eligibility_trace(mem, spike_train, args):
  """
  input:
    mem: membrane potential of i-th neuron for all 
         relevant timesteps, 1-D array shape: (timesteps)
    spike_train: j-th neuron, 1-D array, shape: (timesteps)
    thres: firing threshold
  Returns:
    eligibility trace
  """
  beta = 1 # mV^-1
  h_i = beta*(mem - thres)

  post = 1 / (1 + torch.abs(h_i))**2 
  pre_synaptic_trace = doubleExponential(spike_train, dt, t_rise, t_decay, time)

  hebbian = post * pre_synaptic_trace #hebbian coincidence term

  #synaptic eligibility trace
  eligibility_trace = doubleExponential(hebbian, dt, t_rise, t_decay, time)

  return elibility_trace

### Error Signal

In [ ]:
#@title Output Error Signal

def error_signal(spike_train, desired_spike_train):
  """
  Returns the error signal (time series)
  """
  # output spike train
  filter_1 = doubleExponential(spike_train, dt, t_rise,
                               t_decay, time)
  # desired spike train
  filter_2 = doubleExponential(desired_spike_train, dt, t_rise,
                               t_decay, time)
  
  error = filter_2 - filter_1
  return error



In [ ]:
#@title Feedback Signal


### Per Parameter Learning Rate

Zenke & Ganguli (2018) used a per parameter learning rate. 

In [ ]:
def learning_rate():
  """

  """

### Poisson Spike Trains

Based on the method suggested by David Heeger [here](https://www.cns.nyu.edu/~david/handouts/poisson.pdf)



In [14]:
def Poisson_trains(n, lam, timesteps, dt):
  """

  inputs:
    n - number of poisson spike trains 
    lam - 1-D array containing mean value of poisson trains
  Returns

  """
  trains = torch.zeros((n, timesteps))
  unif = torch.rand((n, timesteps))
  print(unif)
  print(lam*dt)

  counter = 0
  for i in range(n):
    trains[unif <= lam[i]*dt] = 1
    counter += len(unif <= lam[i]*dt)
  print("Total No. of Spikes", counter)

  return trains

In [17]:
trains = Poisson_trains(1, 100*np.ones(10), 1000, 1e-4)

print(trains)

tensor([[4.6941e-01, 9.2857e-01, 1.5926e-01, 3.2097e-01, 7.3991e-02, 8.5742e-01,
         1.4587e-01, 3.3137e-01, 2.6911e-01, 4.5206e-02, 6.2814e-01, 8.9467e-01,
         2.8943e-01, 2.0006e-01, 6.8276e-01, 4.9644e-02, 8.5288e-02, 7.7424e-01,
         8.1146e-01, 8.1954e-01, 9.2050e-01, 7.3686e-01, 5.1565e-01, 6.5492e-01,
         9.7569e-01, 1.6577e-01, 9.8835e-01, 8.2683e-02, 2.7187e-01, 5.7670e-01,
         9.1205e-01, 9.0022e-01, 9.2279e-01, 4.6950e-01, 3.6623e-02, 1.4586e-01,
         2.3351e-01, 2.4276e-01, 4.9419e-01, 3.6346e-01, 6.6140e-01, 7.9350e-01,
         5.5390e-01, 2.0474e-01, 3.1089e-01, 5.3035e-01, 9.4335e-01, 4.4111e-01,
         7.1835e-01, 9.4875e-02, 2.0452e-01, 4.7483e-01, 7.0753e-01, 4.3659e-01,
         2.4363e-01, 7.6736e-02, 9.4806e-01, 9.7339e-01, 9.3424e-01, 3.8554e-01,
         8.4655e-01, 2.1704e-01, 7.0620e-01, 4.7297e-01, 9.5011e-01, 7.8698e-01,
         4.9152e-01, 3.8943e-01, 7.1294e-01, 6.8333e-01, 8.5256e-01, 6.0764e-01,
         9.3572e-01, 4.0508e

In [ ]:
#@title Step Function for Spikes
def spike_fn(x, thres):
  """
  Implements a heaviside function centred at the firing threshold
  """
  x = x - thres
  out = torch.zeros_like(x)
  out[x > 0] = 1
  return out

### Single Neuron Model

In [ ]:
nb_inputs = 100 # 100 spike trains as inputs that repeat every 500 ms
nb_outputs = 1 

batch_size = 256

nb_steps = 5000
timestep_size = 1e-4 # 0.1 msec timesteps


In [ ]:
tau_syn = 
tay_mem = 

alpha = float(np.exp(timestep_size/tau_syn))
beta = float(np.exp(timestep_size/tau_mem))

In [ ]:
#@title Input Spike Trains
input_trains = Poisson_trains(100, )

In [ ]:
#@title Weight Initialization

weight_scale = 7*(1 - beta) # copied from spytorch

weights = torch.empty((nb_inputs, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.init.nn.normal_(weights, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))

In [ ]:
h1 = torch.einsum()

In [ ]:
mem = torch.zeros((batch_size, nb_outputs), device=device, dtype=dtype)
spks = torch.zeros((batch_size, nb_outputs), device=device, dtype=dtype)

mem_rec = []
spks_rec = []

for i in range(epochs):

  for t in range(nb_steps):

    h1 = torch.einsum('abd,dc->abc', ())

    new_spk = alpha*spks + h1[:, t]
    new_mem = beta*mem + 

